## TODO ##
- dodać lime
- wydzielić sensowne fragmenty kodu do plików pythonowych
- odpalić kod na innych datasetach
- odpalić więcej random-searchów
- dodać readme
- odpalić na obciętym datasecie
- wrzucić rezultaty do README.md
- https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

---

- na całym zbiorze danych
- random forest + xgboost
- CNN
- grid search z i be normalizacji

In [1]:
import glob, os
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from collections import Counter

from sklearn.svm import SVR, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn import  metrics

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score


## Load datasets ##

In [4]:
from types import SimpleNamespace 
from typing import Tuple


class Dataset:
    def __init__(self, path):
        self.path = path

        dataset = dict(np.load(self.path, allow_pickle=True))
        
        self.X = dataset.pop('X')
        self.y = dataset.pop('y')
        self.meta = dataset.pop('meta')[()]
        
        assert len(dataset) == 0
        
        self.y_binary = np.where(self.y == None, 0, 1)
    
    def normalize(self, mean=None, std=None):
        # TODO: czy powinniśmy normalizować per współrzędna czy globalnie?
        if mean is None:
            mean = np.mean(self.X, axis=0)
            
        if std is None:
            std = np.std(self.X, axis=0, ddof=1)
            
        # NOTE: inplace operators to prevent memory allocations
        self.X -= mean
        self.X /= std
        
        return mean, std
    
    def sample(self, n, *, balanced=False, with_idx=False, random_state=None) -> Tuple[np.ndarray, np.ndarray]:
        """
        Choice `n` random samples from dataset.
        
        @param n: number of random samples to choose,
        @param balanced: if True number of samples for each class will be aprox. equal,
        @param with_idx: return data indexes of sampled records,
        @param random_state: random state used to generate samples indices,
        """
        assert len(self.y_binary.shape) == 1
        
        if balanced:
            counts = Counter(self.y_binary)
            class_count = len(counts)
            
            # NOTE: https://stackoverflow.com/questions/35215161/most-efficient-way-to-map-function-over-numpy-array/35216364
            probs = np.array([1.0 / (class_count * counts[x]) for x in self.y_binary])
        else:
            probs = None
            
        idx = np.random.RandomState(random_state).choice(self.y.size, size=n, p=probs)
        
        if with_idx:
            return idx, self.X[idx], self.y_binary[idx]
        
        return self.X[idx], self.y_binary[idx]
    
    def frame_to_time(self, frame: int) -> float:
        """Convert frame id to time in sec."""
        return librosa.core.frames_to_time(
            frame,
            sr=self.meta['sampling_rate'],
            hop_length=self.meta['hop_length'],
            n_fft=self.meta['n_fft']
        )

In [6]:
train_data = test_data = None
train_data = Dataset('ch1-2018-11-20_10-29-02_0000012.wav.trimed.npz')
print(train_data.path, Counter(train_data.y_binary))

mean, std = train_data.normalize()
print("mean: %f; std: %f" % (np.mean(train_data.X), np.std(train_data.X, ddof=1)))

#################################################################
# print()
# test_data = Dataset('ch1-2018-11-20_10-26-36_0000010.wav.trimed.npz')
# print(test_data.path, Counter(test_data.y_binary))

# test_data.normalize(mean, std)
# print("mean: %f; std: %f" % (np.mean(test_data.X), np.std(test_data.X, ddof=1)))

ch1-2018-11-20_10-29-02_0000012.wav.trimed.npz Counter({0: 196372, 1: 38013})
mean: 0.000000; std: 1.000209


In [ ]:
test_data.sample(10, with_idx=True, random_state=1)[0]

array([[ 2.8617641e-02,  3.6422275e-02,  6.0354851e-02, ...,
         3.9282343e-01, -7.7947730e-01,  1.7884290e-01],
       [ 9.4728256e-03,  1.6607676e-02, -4.8397966e-03, ...,
        -3.6532238e-02,  2.1068742e-02,  5.7583737e-01],
       [-9.2739612e-03, -1.0576712e-02, -1.9489793e-05, ...,
         3.3974552e-01,  3.7376234e-01,  6.3606781e-01],
       ...,
       [-9.2718471e-03, -7.2237165e-03, -4.6709487e-03, ...,
         4.3271102e-02, -7.5808579e-01, -5.9807819e-01],
       [-2.4606441e-03,  5.1627546e-03, -3.7304866e-03, ...,
         1.5693192e-01, -7.1247154e-01, -3.9950964e-01],
       [ 5.5131242e-02,  9.0585597e-02,  1.7826210e-01, ...,
         2.0846680e-01, -3.6042258e-01, -9.0065479e-02]], dtype=float32)

# SVM classification #

### Correlation matrix ###

In [ ]:
corr = np.corrcoef(train_data.X.T)
print(corr.shape)

plt.figure(figsize=(14,12))
sns.heatmap(corr)

## Sample data for training ##

In [ ]:
N_samples = 5000

X, y = train_data.sample(N_samples, balanced=True, random_state=43)

print(X.shape)
print(Counter(y))

## Test baseline model ##

In [ ]:
def evaluate_model(model, dataset, n=10000, balanced=False):
    idx, X, y_true = dataset.sample(n, balanced=balanced, with_idx=True)
    
    y_pred = model.predict(X)

    print(confusion_matrix(y_true, y_pred))

    print('accuracy:', accuracy_score(y_true, y_pred))
    precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, average='macro')

    print('precision:', precision)
    print('recall:', recall)
    print('fscore:', fscore)
    c = Counter(y_true)
    print('support:', c)
    
    return idx, y_pred, y_true

In [ ]:
# model = SVC(kernel='rbf', gamma='auto') # 0.9021
# model = SVC(kernel='poly', degree=3, gamma='auto') # 0.93335
# model = SVC(kernel='poly', degree=2, gamma='auto') # 0.9414
# model = SVC(kernel='linear', degree=1, gamma='auto') # 0.9402
# model = SVC(kernel='sigmoid', gamma='auto')
model = SVC(kernel='poly', degree=1, gamma='auto') # 0.9424

c = Counter(y)
print(c)
print('Baselines:')
print('weighted random accuracy:', sum(i*i for i in c.values()) / sum(c.values())**2)
print('max random accuracy:', max(c.values()) / sum(c.values()))
print()

model.fit(X, y)
print('Done')

# Evaluation
print('Trainset:')
evaluate_model(model, train_data)

print()
print('TestSet:')
print('unbalanced:')
evaluate_model(model, test_data)
print('balanced:')
evaluate_model(model, test_data, balanced=True)

# Hyperparameters optimization (with RandomizedSearch) #

In [ ]:
#WARNING: brudne haxy
import signal, time, sys, os
import multiprocessing
import random


class FitTimeoutError(Exception):
    pass


def timeout(clf, timeout: float = 120.0):
    """Estimators' decorator for timouting fitting process after `timeout` seconds."""
    def fit(self, *args, _timeout=timeout, **kwargs):
        pool = multiprocessing.pool.ThreadPool(1)
        async_result = pool.apply_async(clf.fit, (self, *args), kwargs)
        try:
            return async_result.get(_timeout)
        except multiprocessing.TimeoutError as e:
            raise FitTimeoutError(f"fit timeout after {timeout:.2f}s") from e
            
    name = f"{clf.__name__}_with_timeout"
    cls = type(name, (clf,), dict(fit=fit))   
    
    setattr(sys.modules['abc'], name, cls)  # HACK: z jakiegoś powodu cls należy do modułu `abc` :shrug:
    
    return cls

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

distributions = dict(
    kernel=['linear', 'rbf', 'poly'],
    degree=[1, 2, 3],
    C=[0.01, 0.1, 0.5, 1, 2, 5, 10, 100],
    gamma=['auto', 100, 10, 1, 0.1, 10, 0.01, 0.001]
)

# NOTE: comment to perform full optimization
distributions = dict(
    kernel=['rbf'], C=[10], gamma=['auto']
)

clf = RandomizedSearchCV(
    timeout(SVC, timeout=120)(), 
    distributions,
    random_state=2,
    verbose=1,
    n_jobs=3,
    cv=5,
    scoring='f1',
    n_iter=60,
    error_score=-np.inf,
    refit=False,
    return_train_score=True
)

print(clf)

clf.fit(X, y)

## Summarize RandomSearchCV ##

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

df = pd.DataFrame(clf.cv_results_)
df = df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score', 'mean_train_score', 'mean_fit_time']]
df.sort_values(by='rank_test_score')


## Best scores for given parameters combinations ##

In [ ]:
import itertools
import seaborn as sns

listables = []
for key, value in clf.param_distributions.items():
    if isinstance(value, list):
        listables.append(key)

for a, b in itertools.combinations(listables, 2):
#     print(a,b)
    params_a = clf.param_distributions[a]
    params_b = clf.param_distributions[b]
    
    results = []
    for val_a, val_b in itertools.product(params_a, params_b):
        crit_a = clf.cv_results_[f'param_{a}'] == val_a
        crit_b = clf.cv_results_[f'param_{b}'] == val_b
        try:
            best = np.max(clf.cv_results_['mean_test_score'][crit_a & crit_b])
        except ValueError:
            best = np.nan 
        results.append(best)
#         print('\t', val_a, val_b, best)
    results = np.array(results).reshape((len(params_a), len(params_b)))

    plt.figure(figsize=(len(params_b), len(params_a)))
    ax = sns.heatmap(results, xticklabels=params_b, yticklabels=params_a, annot=True, fmt=".2f", vmin=0.5, vmax=1.0, cmap='viridis')
    ax.set_facecolor("black")
    ax.set_title(f"{a} vs {b}")
    ax.set(xlabel=b, ylabel=a)

## Fit the best estimator ##

In [ ]:
print(clf.best_params_)
print(clf.best_score_)
best_estimator = clf.estimator.set_params(**clf.best_params_)
best_estimator.fit(X, y, _timeout=100)
print()

print('non-balanced:')
evaluate_model(best_estimator, test_data)
print()

print('balanced:')
idx, y_pred, y_true = evaluate_model(best_estimator, test_data, balanced=True)

In [ ]:
N = 6

################################################
fig, axs = plt.subplots(1, N, figsize=(16, 14))
fig.suptitle('y_true=0 y_pred=1')
for ax, pos in zip(axs, np.random.choice(idx[(y_pred==1) & (y_true == 0)].reshape(-1), N)):
    ax.imshow(test_data.X[pos-20:pos+20].T)
    ax.set_title(f"{pos}")
    ax.axvline(19,color='red')
    ax.axvline(21,color='red')

################################################
fig, axs = plt.subplots(1, N, figsize=(16, 14))
fig.suptitle('y_true=1 y_pred=0')
for ax, pos in zip(axs, np.random.choice(idx[(y_pred==0) & (y_true == 1)].reshape(-1), N)):
    ax.imshow(test_data.X[pos-20:pos+20].T)
    ax.set_title(f"{pos}")
    ax.axvline(19,color='red')
    ax.axvline(21,color='red')


# Generate Selection file #

In [ ]:
from sklearn.metrics import classification_report

%time y_pred = best_estimator.predict(test_data.X)

print(classification_report(test_data.y_binary, y_pred))

print(Counter(y_pred))
print(Counter(test_data.y_binary))

In [ ]:
import pandas as pd

def predict_annotations(dataset, y_pred, *, smoothing=0, min_length=0):
    annotations = []
    
    for i, value in enumerate(y_pred):
        if value == 1:
            if len(annotations) and i - annotations[-1][1] - 1 <= smoothing:
                annotations[-1][1] = i
            else:
                annotations.append([i, i])

    annotations = [(a, b) for a, b in annotations if b - a + 1 >= min_length]
    
    return annotations


def create_selection_table(dataset, annotations):
    return pd.DataFrame.from_records([
        (i, 'Spectrogram 1', 1, dataset.frame_to_time(a), dataset.frame_to_time(b), dataset.meta['audio_name']) for i, (a, b) in enumerate(annotations, 1)
    ], columns=[
        'Selection',
        'View',
        'Channel',
        'Begin Time (s)',
        'End Time (s)',
        'Begin Path',
    ], index='Selection')
    
ann = predict_annotations(test_data, y_pred, smoothing=5, min_length=20)

df = create_annotations_table(test_data, ann)

df.to_csv(test_data.meta['audio_name'].replace('.wav', '.trimed.txt'), sep="\t")
df.head(20)

# TODO: Check predictions with LIME #

- https://pbiecek.github.io/PM_VEE/LIME.html
- https://github.com/pbiecek/InterpretableMachineLearning2018S
- biblioteka: https://github.com/marcotcr/lime

# TODO: Random Forest Classifier #

In [ ]:
# import sys
# import unittest.mock
# import sklearn
# from sklearn.model_selection import RandomizedSearchCV


# def _fit_and_score(estimator, *args, **kwargs):
#     print('Faking fit')
#     orig_fit_and_score(estimtor, *args, **kwargs)

# class RandomizedSearchCVWithTimeout(RandomizedSearchCV):
#     def __init__(self, estimator, param_distributions, n_iter=10, scoring=None,
#                  n_jobs=None, iid='deprecated', refit=True,
#                  cv=None, verbose=0, pre_dispatch='2*n_jobs',
#                  random_state=None, error_score=np.nan,
#                  return_train_score=False, timeout=None):
#         super().__init__(estimator=estimator, param_distributions=param_distributions, 
#                          n_iter=n_iter, scoring=scoring, n_jobs=n_jobs, iid=iid, refit=refit,
#                          cv=cv, verbose=verbose, pre_dispatch=pre_dispatch,
#                          random_state=random_state, error_score=error_score,
#                          return_train_score=return_train_score)
        
#         self._timeout = timeout
#     def fit(self, *args, **kwargs):
#         orig_fit_and_score = sklearn.model_selection._validation._fit_and_score
        
#         def timeouted_fit(estimator):
#             orig = estimator.fit
#             return orig
        
#         def _fit_and_score(estimator, *args, **kwargs):
#             print('Faking fit')
#             orig_fit_and_score(estimtor, *args, **kwargs)
# #             with unittest.mock.patch(estimator.fit, timeouted_fit(estimator)):
# #                 orig_fit_and_score(estimator, *args, **kwargs)
        
        
#         with unittest.mock.patch('sklearn.model_selection._validation._fit_and_score', _fit_and_score):
#             super().fit(*args, **kwargs)
            
            
# # #     @property
# # #     def verbose(self):
# # #         frame = sys._getframe(1)
# # #         print(dir(frame))
# # #         if str(frame.f_code.co_filename).endswith('sklearn/model_selection/_search.py'):
# # #             print('-'*20, frame.f_lineno)
# # #             print(frame.f_code.co_filename)
# # #             print('>>', frame.f_locals.keys())
# # # #         print(sys._getframe(1).f_code.co_filename)
# # # #         print('*'*100)
# # #         return self._verbose
# # #     @verbose.setter
# # #     def verbose(self, value):
# # #         self._verbose = value
    
# # #     def _run_search(self, evaluate_candidates):
# # #         print('run_search')
# # #         raise RuntimeError()
        
# # print(RandomizedSearchCVWithTimeout.mro())

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# parameters = {
#     'n_estimators': [100, 500, 1000],
#     'max_depth': [20, 40, 100],
# }

# rf = RandomForestClassifier(random_state=0)

# clf = GridSearchCV(rf, parameters, cv=5, verbose=10)

# clf.fit(X, y_binary)

# # clf.cv_results_
# print(clf.best_params_, clf.best_score_)

In [ ]:
# print(clf.best_params_, clf.best_score_)

In [ ]:
# idx, y_pred, y_true = evaluate_model(rf, train_data)

## Jak oceniać modele? jaka miara? ##
## Jak normalizować dane? potrzebne dla SVMa ##